# Torchtext Tutorial
- 기능:
    - 파일 로드
    - 토크나이징
    - 단어 집합(Vocab)
    - 정수 인코딩(Integer encoding): 전체 코퍼스의 단어들 각각 고유한 정수로 맵핑
    - 단어 벡터(Word Vector): 단어 집합의 단어들에 고유한 임베딩 벡터를 만들어 준다. 랜덤값으로 초기화 한 값일수도 있고, 사전 훈련된 임베딩 벡터를 로드할 수도 있다.
    - 배치화: 훈련 샘플들의 배치 만들어 줌. 이 과정에서 패딩도 이뤄진다.

## 데이터 불러오기

In [71]:
import urllib.request
import pandas as pd

In [72]:
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

In [ ]:
# country 컬럼을 선택합니다.
# 컬럼의 값과 조건을 비교합니다.
# 그 결과를 새로운 변수에 할당합니다.
is_venezuela = train_df['country'] == 'Venezuela'

# 조건를 충족하는 데이터를 필터링하여 새로운 변수에 저장합니다.
venezuela = df[is_venezuela]

# 결과를 출력합니다.
venezuela

## 필드 정의하기(torchtext.data)
- sequential: 시퀀스 데이터 여부
- use_vocab: 단어 집합 만들 것인지 여부
- tokenize: 어떤 토큰화 함수 사용할 것인지 지정
- lower: 소문자화
- batch_first: 미니 배치 차원을 맨 앞으로 해 데이터 불러올 것인지 여부
- is_target: 레이블 데이터 여부
- fix_length: 최대 허용 길이. 이 길이에 패딩 진행

In [73]:
from torchtext.legacy import data

# 필드 정의
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=str.split,
                  lower=True,
                  batch_first=True,
                  fix_length=300)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True)

## 데이터셋 만들기
- TabularDataset: 데이터 불러오면서 필드에서 정의했던 토큰화 방법으로 토큰화 수행

In [74]:
from torchtext.legacy.data import TabularDataset

train_data, test_data = TabularDataset.splits(path='.', train='./data/train.csv', test='./data/test.csv', format='csv',
                                             fields=[('text', TEXT), ('label', LABEL)], skip_header=False)

In [75]:
print('training: {}'.format(len(train_data)))
print('test: {}'.format(len(test_data)))

training: 25000
test: 25000


In [76]:
# 주어진 인덱스의 샘플 확인 가능
print(vars(train_data[0]))

{'text': ['for', 'a', 'movie', 'that', 'gets', 'no', 'respect', 'there', 'sure', 'are', 'a', 'lot', 'of', 'memorable', 'quotes', 'listed', 'for', 'this', 'gem.', 'imagine', 'a', 'movie', 'where', 'joe', 'piscopo', 'is', 'actually', 'funny!', 'maureen', 'stapleton', 'is', 'a', 'scene', 'stealer.', 'the', 'moroni', 'character', 'is', 'an', 'absolute', 'scream.', 'watch', 'for', 'alan', '"the', 'skipper"', 'hale', 'jr.', 'as', 'a', 'police', 'sgt.'], 'label': '0'}


In [77]:
# 필드 구성 확인
print(train_data.fields.items())

dict_items([('text', <torchtext.legacy.data.field.Field object at 0x160c9bb90>), ('label', <torchtext.legacy.data.field.Field object at 0x160c9bb50>)])


## 단어 집합(Vocabulary) 만들기

In [78]:
TEXT.build_vocab(train_data, min_freq=10, max_size=10000)
print('단어 집합의 크기 : {}'.format(len(TEXT.vocab)))


단어 집합의 크기 : 10002


In [83]:
LABEL.build_vocab(train_data)
print(LABEL.vocab.stoi)

defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x189b81ed0>>, {'<unk>': 0, '0': 1, '1': 2})


## torchtext의 데이터로더 만들기
- 데이터로더는 데이터셋에서 미니배치만큼 데이터 로드하게 만들어 주는 역할
    - torchtext에서는 iterator 사용해 데이터로더를 만든다.

In [64]:
from torchtext.legacy.data import Iterator

In [65]:
batch_size = 5

train_loader = Iterator(dataset=train_data, batch_size=batch_size)
test_loader = Iterator(dataset=test_data, batch_size=batch_size)

In [66]:
# 25,000개의 샘플을 배치 크기 5개씩 묶어주었기 때문
print('훈련 데이터의 미니 배치 수 : {}'.format(len(train_loader)))
print('테스트 데이터의 미니 배치 수 : {}'.format(len(test_loader)))

훈련 데이터의 미니 배치 수 : 5000
테스트 데이터의 미니 배치 수 : 5000


In [67]:
# 첫번째 미니배치 가져와서 batch라는 변수에 저장해보자
batch = next(iter(train_loader))

In [68]:
print(batch.text[0])

tensor([  10,   20,    7,    3, 2274,   77,   12, 1113,   17,    0,   18, 6660,
           0,   13,  165,    0,  935,   11,  281,   12,    7, 6285,   15,    3,
        4693, 3052, 1131,    4, 8079, 9739,    7, 1001,   12, 1113,   29,    3,
           0,    0,  266,    0, 6236,   44,   23,   43, 9496,  580,    0,   40,
        1588,    0,   25,  207,   28, 3980,    4, 1610, 1269,    2, 1719,  629,
          44,    0,   77,  189,   48,    2,  258,  122,  699,    2,  548,   41,
           6,  110,  101,    4,   98,   15,   10,   75,    0,   13,  165,   74,
          23,  184,    0,   77,   11, 6294,    2,  249,    5, 2152,    2,  131,
           7,   48,  788,  308,   34,    2,   80,    0,   32,  952,    0, 2026,
        3160,  689,    4, 6923, 4604,   19,   16,    3, 4693, 3052, 1764,   39,
         135, 1415,    4, 4869, 2244,    4,    0, 1052,    0,  108,   13,    0,
          12,   34, 3587,   13,  255,   20,  458,   29,    3, 2922,    0,   17,
           3, 1166,    0,    5, 4693,   

## <pad> 토큰이 사용되는 경우

In [69]:
len(LABEL.vocab)

AttributeError: 'Field' object has no attribute 'vocab'